## Hello Machine Learning

Train a simple logistic regression model to predict tumor vs. normal from gene expression data

In [3]:
import os
import requests
import pandas as pd

In [10]:
%%time
if not os.path.exists("/scratch/tcga_target_gtex.h5"):
    print("Downloading Wrangled TCGA, TARGET and GTEX training data")
    r = requests.get("https://bme230badmindiag811.blob.core.windows.net/data/tcga_target_gtex.h5", stream=True)
    r.raise_for_status()
    with open("/scratch/tcga_target_gtex.h5", "wb") as f:
        for chunk in r.iter_content(chunk_size=32768):
            f.write(chunk)

CPU times: user 19.7 s, sys: 18 s, total: 37.6 s
Wall time: 3min 51s


In [12]:
X = pd.read_hdf("/scratch/tcga_target_gtex.h5", "expression")
X.head()

,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,snoU2-30,snoU2_19,snoU83B,snoZ196,snoZ278,snoZ40,snoZ6,snosnR66,uc_338,yR211F11.2
GTEX-1117F-0226-SM-5GZZ7,-9.9658,-9.9658,-9.965800,4.4595,0.9343,-5.0116,7.5126,0.8164,-2.1140,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,0.158328,-9.9658
GTEX-1117F-0426-SM-5EGHI,-9.9658,-9.9658,-9.965800,1.1512,-1.2828,-6.5064,6.0777,-2.3147,0.5568,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.126224,-9.9658
GTEX-1117F-0526-SM-5EGHJ,-9.9658,-9.9658,-9.965800,5.2411,0.8488,-6.5064,10.0319,0.1257,-1.1172,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-0.864951,-9.9658
GTEX-1117F-0626-SM-5N9CS,-9.9658,-9.9658,-9.965800,5.4758,2.6325,-9.9658,9.7572,1.7702,-1.8836,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-0.918501,-9.9658
GTEX-1117F-0726-SM-5GIEN,-9.9658,-9.9658,-3.406063,4.5534,1.3051,-9.9658,7.7931,-0.0725,-2.2447,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.018394,-9.9658


In [13]:
Y = pd.read_hdf("/scratch/tcga_target_gtex.h5", "labels")
Y.head()

,category,disease,primary_site,sample_type,gender,study,tumor_normal
id,,,,,,,
GTEX-1117F-0226-SM-5GZZ7,Adipose - Subcutaneous,Adipose - Subcutaneous,Adipose Tissue,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0426-SM-5EGHI,Muscle - Skeletal,Muscle - Skeletal,Muscle,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0526-SM-5EGHJ,Artery - Tibial,Artery - Tibial,Blood Vessel,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0626-SM-5N9CS,Artery - Coronary,Artery - Coronary,Blood Vessel,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0726-SM-5GIEN,Heart - Atrial Appendage,Heart - Atrial Appendage,Heart,Normal Tissue,Female,GTEX,Normal
